<a href="https://colab.research.google.com/github/kreatorkat2004/kreatorkat2004/blob/main/comp341_hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## COMP 341: Practical Machine Learning
## Homework Assignment 6: Wrapping it all up
### Due: Tuesday, November 25 at 11:59pm on Gradescope

This assignment will first walk through some of the fundamentals of PyTorch as we build a deep learning classifier to identify items of clothing from black and white images, then revisit Homework 4, where we used regression-based methods to predict the list prices of homes in the Houston area. The dataset that we will be using in the first portion, [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist), is often used for benchmarking and is inspired by the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset of handwritten digits. For the second part, we will supplement our prediction task with also the first picture on the house listing.

As always, fill in missing code following `# TODO:` comments or `####### YOUR CODE HERE ########` blocks and be sure to answer the short answer questions marked with `[WRITE YOUR ANSWER HERE]` in the text.

All code in this notebook will be run sequentially so make sure things work in order! Be sure to also use good coding practices (e.g., logical variable names, comments as needed, etc), and make plots that are clear and legible.

For this assignment, there will be **15 points** allocated for general coding and formatting points:
* **5 points** for coding style
* **5 points** for code flow (accurate results when everything is run sequentially)
* **5 points** for additional style guidelines listed below

Additional style guidelines:
* **The ipynb files are not rendering properly on gradescope due to size limits, so for the convenience of your TAs, please export a pdf of your colab notebook (and include a rice-accessible private link to the notebook at the end of the assignment). Your file should be named: `netid-hw6.pdf`**
* For any TODO cell, make sure to include that cell's output in the .ipynb file that you submit. Many text editors have an option to clear cell outputs which is useful for getting a blank slate and running everything beginning-to-end, but always be sure to run the notebook before submitting and ensure that every cell has an output.
* When displaying DataFrames, please do not include `.head()` or `.tail()` calls unless asked to. Just removing these calls will work as well, and will allow us to see both the beginning and end of your DataFrames, which help us ensure data is processed properly. Notebooks will by default show only the beginning and end, so you don't have to worry about long outputs here.
* If column names are specified in the question, please use the specified name, and please avoid any sorting not specified in the instructions.
* For plots, please ensure you have included axis labels, legends, and titles.
* To format your short answer responses nicely, we recommend either **bolding** or *italicizing* your answer, or formatting it ```as a code block```.
* Generally, please keep your notebook cells to one solution per cell, and preserve the order of the questions asked.
* Finally, this can be harder to check/control and depends on which plotting libraries you prefer, but it would be helpful to limit the size/resolution of plot images in the notebook. Our grading platform has an upper limit on submission sizes it can display, and high-res plots are the usual culprit when submissions are hidden or truncated.

### Setup
First, we need to import the necessary libraries for this assignment.

In [ ]:
import pandas as pd
import numpy as np

from PIL import Image, ImageOps
import os

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from collections import defaultdict

Packages such as [TorchMetrics](https://torchmetrics.readthedocs.io/en/stable/) provide more options for evaluation metrics than what PyTorch natively provides. Here, we install the package and load their implementation of MSE, since it natively supports RMSE (which we will use in Part 2 instead of implementing it from scratch).

In [ ]:
!pip install torchmetrics

In [ ]:
from torchmetrics import MeanSquaredError

Add additional modules/libraries to import here (rather than wherever you first use them).

In [ ]:
# additional modules / libraries to import


Make sure we are using the latest torch version (PyTorch 2.8) and check if we have GPUs available (cu or cuda vs cpu).

In [ ]:
torch.__version__

We would preferably like to also take advantage of any gpu resources to speed up our training. To this end, in Colab, you can click on the RAM / Disk monitoring bars towards the upper right corner of your Colab instance. Then in the menu that appears, click "Change runtime type" and select "GPU" under "Hardware accelerator." Let's check if the GPU is indeed detected. Note that whis code snippet will by default assume we are using CPU resources unless it can detect GPUs available.

In [ ]:
device = "cpu"
if (torch.cuda.is_available()):
  device = "cuda"
print("device: " + device)

The `device` string simply will be used in a few key parts of the code to signal to PyTorch whether or not to move the model / weights etc to the GPU, if available. Using GPUs for this assignment is optional (though doing so will allow you to train faster).

As in previous assignments, we will use Google Drive to access the dataset that you will use in the second half of the homework. However, there are some slight modifications once more because your `DataLoader` will be frequently accessing the raw image files, which can be very slow if it is having to access the files on Google Drive constantly.

By now, you will probably already created your local `comp341` directory:
1. Go to 'My Drive' in your own Google Drive
2. Make a new folder named `comp341`

Now, we will copy a shortcut to a zipped data file with all of the images and metadata for the houses:

3. From the [Google Drive link](https://drive.google.com/file/d/1H3QvduNrpfEIn6Q4ODEc6gpnr9eiysLJ/view?usp=sharing), you can right click the `comp341-hw6.zip` file, and select `Add shortcut to Drive`, and add a link to the file to your `comp341` folder. This is a convenient alternative to having to download and re-upload the files to your own drive.

If you run into trouble with accessing the files from the shortcut, then:

4. Download the `comp341-hw6.zip` file and click `New -> File Upload` to upload the downloaded file from your computer.

Now, we will mount your Google Drive as usual, but we will have an additional step of copying over the zip file to your local colab instance so that files can be accessed quickly.

In [ ]:
# note that this command will trigger a request from google to allow colab
# to access your files: you will need to accept the terms in order to access
# the files this way
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# if you followed the instructions above exactly, your zipped data file should
# be located at the file path below; if your files are in a different directory
# on your Google Drive, you will need to change the path below accordingly
ZIPPATH = '/content/drive/My Drive/comp341/comp341-hw6.zip'

In [ ]:
!cp "{ZIPPATH}" .
!unzip -q "comp341-hw6.zip"
!rm "comp341-hw6.zip"

In your local colab instance, you should now have a `house_imgs/` directory with many images of homes (includes images from both the training and test sets), as well as two csv files: `home_data_train.csv` and `home_data_test.csv`.

### Part 1: Simple CNN using FashionMNIST [28 pts]

PyTorch provides convenient functionality to download and load Fashion-MNIST as a `Dataset` that can be used with `DataLoader`!

In [ ]:
# we can load the data directly using PyTorch
# this will automatically download the data into your colab folder (and redownload as necessary)
fm_data_train = torchvision.datasets.FashionMNIST('./data', download=True,
                                                  transform=transforms.Compose([transforms.ToTensor()]))
fm_data_valid = torchvision.datasets.FashionMNIST('./data', download=True, train=False,
                                                  transform=transforms.Compose([transforms.ToTensor()]))

In [ ]:
# based on the label definitions given by the original dataset
# https://github.com/zalandoresearch/fashion-mnist#labels
# we have provided a convenient function to get the actual labels
# corresponding to each numeric label in the dataset
def label_name(label):
  label_mapping = {
      0: "T-shirt/Top",
      1: "Trouser",
      2: "Pullover",
      3: "Dress",
      4: "Coat",
      5: "Sandal",
      6: "Shirt",
      7: "Sneaker",
      8: "Bag",
      9: "Ankle Boot"
      }
  label_num = (label.item() if type(label) == torch.Tensor else label)
  return label_mapping[label_num]

In [ ]:
# we can now loop through a couple of the first few images in our dataset
# note that we enlarge the image simply for easy visualization purposes

tens_to_img = transforms.ToPILImage()
for i in range(4):
  print('label id:', fm_data_train[i][1], 'name:', label_name(fm_data_train[i][1]))
  display(tens_to_img(fm_data_train[i][0]).resize((150,150)))

In [ ]:
# TODO: how many observations are there in your training and validation sets? [1 pt]


In [ ]:
# TODO: make data loaders for the training and validation datasets
# that we loaded above; for now, use a batch size that will allow you to
# use Stochastic Gradient Descent in the training [2 pts]


Based on the data description, we know that there are 10 labels (with mappings between number and actual category provided by our `label_name` method). Let's use the DataLoaders that we just made to count up the number of observations we have per label.

In [ ]:
# TODO: loop through the entire dataset using the DataLoaders you constructed earlier
# and count how many observations we have per label (in both training and validation sets)
# show the results using the label name (not simply the number representation) [1 pts]


We will now construct our model using CNNs with a multilayer perceptron at the end for classification. As the data description shows (and we could also verify easily), each image is grayscale, 28 x 28 pixels.

The TODOs below describe the details of the architecture. Implementaiton-wise, you are free to declare the full architecture in `__init__` or a combination of `__init__` and `forward`. As we discussed in class, there are multiple ways to implement the same architecture. Feel free to opt for the one you feel is most intuitive.

In [ ]:
# first model attempt
class StylishNN(nn.Module):
  def __init__(self, num_classes=10):
    # TODO: inherit from nn.Module [1 pt]


    # TODO: add a first convolution layer together with a ReLU activation function
    # for this first convolution layer, we want to apply 12 filters, where each
    # filter is 5x5, taking strides of 1, and padded to maintain the original image size
    # follow up on this convolution with max pooling with a 2x2 filter and stride 2 [2 pts]



    # TODO: add a second convolution layer that expands the number of filters to 32
    # the filters will still be 5x5 and taking strides of 1, but this time, use a 1 pixel padding instead
    # follow up on this convolution with max pooling again with a 2x2 filter and stride 2 and a ReLU activation function [2 pts]



    # TODO: for the classification portion of this neural network, make 3 fully connected layers
    # you will need to calculate what the current dimensions of your tensors are after convolution
    # and pooling, as you the fully connected layers will use the flattened values
    # with that number as your initial input, here are the number of neurons for each layer, together
    # with which activation function we would like to use [3 pts]
    # fully connected layer 1: 600 neurons, ReLU activation
    # fully connected layer 2: 120 neurons, ReLU activation
    # fully connected layer 3 (output layer): num_classes neurons, no additional activation function,
    # use the log(softmax) of these values to correspond to a probability for each class



  def forward(self, x):
    # TODO: forward pass for the convolution and pooling layers [1 pt]


    # TODO: flatten parameters into 1 dimension before passing them
    # on to our fully connected layers [1 pt]


    # TODO: forward pass for the fully connected layers [1 pt]


    # TODO: return the log(softmax) predictions for each class [1 pt]



Now that we've constructed our model, it is time to train. Instead of stochastic gradient descent, we would prefer to use mini-batch gradient descent. For the next parts, set up your `DataLoader`s to use batch sizes of 64.

In [ ]:
# TODO: make data loaders for the training and validation datasets
# with a batch size of 64; remember that we want to randomize the order that we see our data! [1 pts]


In [ ]:
# TODO: fill in the steps that we want to take within the training loop
# (this function will run once per epoch) [3 pts]
# we have provided some helper code for logging progress

def train(model, train_loader, opt, epoch, verbose=False):
  if verbose:
    print("starting epoch", epoch)
  train_loss = 0
  for i, (image, label) in enumerate(train_loader):
    image, label = image.to(device), label.to(device)

    # TODO: a) forward pass


    # TODO: b) calculate loss


    # TODO: c) backward pass


    # TODO: d) update weight estimates


    # TODO: e) reset gradients to zero


    # we are tracking the sum of losses to calculate the average training loss for this epoch
    train_loss += loss.item()

    # when verbose is on, we will show how the loss is changing across batches within one epoch
    if verbose and ((i % 100) == 0):
      print('training [epoch {}: {}/{} ({:.0f}%)] loss: {:.6f}'.format(
          epoch, i * len(image), len(train_loader.dataset),
          100. * i / len(train_loader), loss.item()))

  avg_tl = train_loss / (i+1)
  print('epoch {} avg training loss: {:.6f}'.format(epoch, avg_tl))
  return avg_tl

In [ ]:
# we have provided the below code for reporting loss and accuracy
# on your validation set, so simply run this snippet without any changes
def valid(model, valid_loader):
  valid_loss = 0
  correct = 0
  with torch.no_grad():
    for i, (image, label) in enumerate(valid_loader):
      image, label = image.to(device), label.to(device)

      # get the prediction
      pred = model(image)

      # get loss
      valid_loss += loss_fn(pred, label).item()

      # calculate the accuracy for this batch
      _, pred = torch.max(pred.data, 1)
      correct += torch.sum(label==pred).item()

  # get the loss for the epoch
  avg_vl = valid_loss / (i+1)
  print('avg validation loss: {:.6f}, accuracy: {}/{} ({:.0f}%)'.format(
      avg_vl, correct, len(valid_loader.dataset),
      100. * correct / len(valid_loader.dataset)))

  return avg_vl

In [ ]:
# we can now initialize the model you designed above
# and optionally put it on the GPU if we have enabled it
# remember to rerun this if you want to re-initialize your parameters!
# (otherwise, the model will simply keep updating the previous parameters)
model = StylishNN().to(device)

# TODO: initialize the optimizer with learning rate 0.01 [1 pt]


# TODO: initialize cross entropy as your loss function [1 pt]



In [ ]:
# now that you have defined your model and set up the training loop
# we can run through 15 epochs and see how our training and validation
# losses change over time (you can just run this code directly)
# note that we have turned on verbose here to better see intermediate progress
# but if the output is overwhelming, you can feel free to turn it off
epoch_list = []
train_loss = []
valid_loss = []

epochs = 15
for e in range(1, epochs+1):
  epoch_list.append(e)
  train_loss.append(train(model, train_loader, opt, e, verbose=True))
  valid_loss.append(valid(model, valid_loader))

Now, we will plot the training and test over the different epochs as well as look for any confusion between classes.

In [ ]:
# TODO: as we trained our model, we captured the epoch,
# average loss for the training and validation sets in
# epoch_list, train_loss, and valid_loss
# use these to plot the loss vs epochs for the training and test set [2 pts]



**Short Answer Question:** What does this epoch vs loss plot tell you about your model and predictions? [1 pt]

`[WRITE YOUR ANSWER HERE]`

In [ ]:
# TODO: use sklearn's convenient confusionmatrix and ConfusionMatrixDisplay
# methods to calculate and visualize the confusion matrix for your
# current predictions, making sure to display the human-readable
# class labels (instead of 0-9) [2 pts]



**Short Answer Question:** Does our model have any trouble distinguishing any items of clothing? If so, which ones does it tend to mixed up? [1 pt]


`[WRITE YOUR ANSWER HERE]`

### Part 2: List Price Revisited [27 pts]

In the ```Setup``` section, you should have copied over the house images and tabular data into your colab instance.

Since we explored this task in Homework 4, we are already somewhat familiar with the tabular features, including some of the data cleaning / preprocessing steps that might be valuable, as well as which ones may be more or less valuable to keep.

Now that we also have some sense of what the house listing images are like based on Part 1, we will set up a regression framework that can use both the tabular features and image features. Along the way, we will also see how our predictions may change depending on what data we use.

We provide several helper functions for setting up the data and functionality to visualize individual examples, which can sometimes be helpful to get a sense of what the model is doing.

In [ ]:
# torch converts the 0-255 RGB values to 0-1 tensors, but it can
# also be beneficial to also standardize the values (or, as we
# see here, subtract the mean RGB values from the images)

# these transformations below help facilitate this
# inv_normalize is provided mainly for visualization sake, so that
# we can flip the standardization process to see the image in its
# original colors

house_mean = [0.5230, 0.5416, 0.4989]
# house_sd = [0.2271, 0.2162, 0.2640]
# only subtracting mean and not also dividing by standard deviation
# can actually sometimes work better, which is what we are doing here
house_sd = [1, 1, 1]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(house_mean, house_sd)
])

inv_normalize = transforms.Normalize(
   mean= [-m/s for m, s in zip(house_mean, house_sd)],
   std= [1/s for s in house_sd]
)


In [ ]:
# convenient function for displaying images
# by default, will reverse the standardization calculation so that we can
# see the images in a "normal" color scheme
def display_data(d, inv_norm=True):
  if isinstance(d['houseid'], list): # we can handle a list of houses
    batch_size = len(d['houseid'])
    for i in range(batch_size):
      if 'price' in d:
        print('price:', "${:,.0f}".format(d['price'][i]))

      if inv_norm:
       display(transforms.ToPILImage()(inv_normalize(d['image'][i])))
      else:
        display(transforms.ToPILImage()(d['image'][i]))
  else: # only an individual house to be displayed
    if 'price' in d:
      print('price:', "${:,.0f}".format(d['price']))

    if inv_norm:
      display(transforms.ToPILImage()(inv_normalize(d['image'])))
    else:
      display(transforms.ToPILImage()(d['image']))

As you may recall, data handling in PyTorch relies heavily on the `Dataset` class. In Part 1, we used one of the pre-built `Datasets`, but most of the time, we need to set up our own. Here, we provide a scaffold `HouseImagesDataset` class.

You will need to fill in any cleaning / data preprocessing steps for the features in `__init__`. We provide code for `__getitem__`, but you will also need to fill in the other method we typically define for a custom `Dataset`: `__len__`.

In [ ]:
class HouseImagesDataset(Dataset): # [4 pts]
    def __init__(self, annot_file, image_dir, train=True):
        # the annotation file is tidy, aka each row is a unique observation in the dataset,
        # but it is not yet clean, which you will address in the TODO below
        df = pd.read_csv(annot_file)

        # TODO: cleaning / preprocessing of features in df



        # TODO: fill in this feature_cols list with the column names of
        # features you would like to use to predict list price (many of the columns
        # will likely be transformed from the original data in annot_file)
        self.feature_cols = []

        self.house_annot = df
        self.image_dir = image_dir
        self.train=train

    def __len__(self):
        # TODO: fill in this method (replacing pass) to return the length of the dataset


    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # we have provided code that will load and transform the indexed ("ith") image
        # as well as features specified earlier in self.feature_cols within the processed
        # pandas DataFrame
        img = Image.open(os.path.join(self.image_dir, self.house_annot.loc[idx, 'houseid'] + '.jpg'))
        img = transform(img)

        features = self.house_annot.loc[idx, self.feature_cols]
        features = features.tolist()
        features = torch.as_tensor(features, dtype=torch.float)

        # depending on whether the Dataset is in training mode, we will have the price data or not
        if self.train:
            item = {'image': img,
                    'houseid': self.house_annot.loc[idx, 'houseid'],
                    'features': features,
                    'price': torch.as_tensor(self.house_annot.loc[idx, 'list_price'], dtype=torch.float)}
        else:
            item = {'image': img,
                    'houseid': self.house_annot.loc[idx, 'houseid'],
                    'features': features}

        return item

In [ ]:
# TODO: initialize the house dataset using the training data you were provided and check the length of the dataset [1 pt]


In [ ]:
# TODO: check that the data loads properly by calling the provided display_data function with a specifically indexed
# item in your house dataset (e.g., house_dataset[3]) [1 pt]


In [ ]:
# TODO: use the convenient torch.utils.data.random_split function to split your loaded dataset into training and
# validation portions, using 75% of the data for training and 25% of the data for validation [1 pt]


We explored how to set up a CNN with several convolutional layers and some fully connected layers in Part 1 when we have image data. Now, we have *both* image and tabular features. One natural way we can handle this is to set up several convolutional layers to extract important image-related features, and in parallel, pass our tabular features that we indicated we want to use (`self.feature_cols` in our `HouseImagesDataset`) through a simple MLP, then simply **concatenate** the two sets of features together before passing them through a set of linear layers before reaching our final prediction.

Recall that with house price predictions, we want to end up with a single non-zero prediction for each house.

In [ ]:
class HybridHouseNN(nn.Module):
  def __init__(self):
    super().__init__()
    # TODO: set up some convolutional layers
    # it is your choice as to how many convolutional blocks, as well as
    # specifics within the blocks: activation function, pooling, etc [1 pt]


    # TODO: set up an MLP for the tabular features that you will be
    # inputting into this model [1 pt]


    # TODO: set up the final set of fully connected layers that
    # takes as input the concatenated set of flattened convolution features
    # together with the output of the MLP from the tabular features
    # to eventually output a single non-negative prediction [1 pt]


  def forward(self, ximg, xfeats):
    # TODO: write out the forward pass steps
    # note that forward now has 2 inputs because we are using both
    # images and non-image features separately at first, before
    # merging them together for the final set of predictions
    # Note: you may also need to adjust the shape of your final prediction
    # so that it plays nice with the loss function etc. [2 pts]




Before training our model, we want to also set up some additional models to what the differences might be if we use *only* images or *only* the tabular features for our predictions. Of course, if we set the models up differently with different hyperparameters, we really cannot have a truly equivalent comparison, but we will try to keep as many of the model blocks the same as possible.

In [ ]:
class HouseImageOnly(nn.Module):
  def __init__(self):
    # TODO: set up this model to only use images for predictions,
    # using the same convolutional blocks that you designed for
    # your HybridHouseNN model; also use a similar set of
    # fully connected layers (changing only the input dimension
    # since you will no longer be concantenating other features) [2 pts]


  def forward(self, ximg):
    # TODO: the forward pass will now only have the image as input [1 pt]


In [ ]:
class HouseFeatsOnly(nn.Module):
  def __init__(self):
    # TODO: set up this model to only use the tabular, non-image features
    # this model will simply be a MLP with those features as input
    # once again, try to keep the layers relatively comparable to what
    # you chose in HybridHouseNN, though of course the dimensionality going
    # into the last set of fully connected layers will likely change drastically
    # without image features [2 pts]


  def forward(self, xfeats):
    # TODO: the forward pass will now only have non-image features as input [1 pt]



As noted in the Setup section, will use RMSE for our loss function:

In [ ]:
loss_fn = MeanSquaredError(squared=False).to(device)

Now, let's fill in the details of the training and validation methods.

In [ ]:
def train(model, train_loader, opt, epoch, mode="both", verbose=False):
  # mode can be "both", "image", or "features", depending on if we are using
  # our HybridHouseNN, HouseImageOnly, or HouseFeatsOnly model
  # we will assume that the model passed to this function matches the mode,
  # and mode will affect whether the model uses image, features, or a combination
  # as input to get the predictions in the forward pass

  if verbose:
    print("starting epoch", epoch)
  train_loss = 0
  for i, batch in enumerate(train_loader):
    image, features, price = batch['image'].to(device), \
                             batch['features'].to(device), \
                             batch['price'].to(device)

    model.train(True)

    # TODO: fill in the code for each of the steps in the
    # training loop, remembering that we want to account for
    # different modes in the forward pass step [2 pts]


    model.train(False)
    if verbose and ((i % 20) == 0):
      print('training [epoch {}: {}/{} ({:.0f}%)] loss: {:.6f}'.format(
          epoch, i * len(image), len(train_loader.dataset),
          100. * i / len(train_loader), loss.item()))

  avg_tl = train_loss / (i+1)
  print('epoch {} avg training loss: {:.6f}'.format(epoch, avg_tl))
  return avg_tl

In [ ]:
def valid(model, valid_loader, mode="both"):
  # as in train, mode can be "both", "image", or "features", depending on if we are using
  # our HybridHouseNN, HouseImageOnly, or HouseFeatsOnly model
  # we will assume that the model passed to this function matches the mode,
  # and mode will affect whether the model uses image, features, or a combination
  # as input to get the predictions in the forward pass
  valid_loss = 0
  correct = 0
  with torch.no_grad():
    for i, batch in enumerate(valid_loader):
      image, features, price = batch['image'].to(device), \
                               batch['features'].to(device), \
                               batch['price'].to(device)

      # TODO: fill in code to calculate pred (the prediction), paying attention to
      # different usage of the model depending on the inputted mode variable [1 pt]


      # get loss
      valid_loss += loss_fn(pred, price).item()


  # get the loss for the epoch
  avg_vl = valid_loss / (i+1)
  print('avg validation loss: {:.6f}'.format(avg_vl))

  return avg_vl

In [ ]:
# TODO: initialize your training and validation DataLoaders, using a batch_size of 64
# and remembering to randomize the order the data points are presented to the model [1 pt]



In [ ]:
# we will use these simple dictionaries to keep track of the loss for our 3 different models
# note that depending on how complex your 3 models are, training may take some time, on the scale
# of 10-30 minutes
# with verbose on below, you should be seeing continual progress during training: if not, then
# double check that you are using GPUs and the image files locally within your colab instance
epoch_list = defaultdict(list)
train_loss = defaultdict(list)
valid_loss = defaultdict(list)

epochs = 30
modes = {'both': HybridHouseNN(), 'image': HouseImageOnly(), 'features': HouseFeatsOnly()}

for m in modes:
  model = modes[m].to(device)
  # TODO: initialize the optimizer (and associated hyperparameters like learning rate) of your choice [1 pt]
  opt = optim.#OPTIMIZER

  print("Current mode:", m)
  for e in range(1, epochs+1):
    epoch_list[m].append(e)
    train_loss[m].append(train(model, train_loader, opt, e, m, verbose=True))
    valid_loss[m].append(valid(model, valid_loader, m))

In [ ]:
# TODO: on a single plot, plot the training and validation losses for your 3 different models [2 pts]


**Short Answer Question:** Based on the results that you see in your plot, what do you think about using images, tabular features, or both for predicting list price? (Of course, the results and associated interpretations may change depending on how the models are set up, so describe your interpretations solely based on the plot that you generated above and not other potential possibilities.) [2 pts]

`[WRITE YOUR ANSWER HERE]`

### Part 3: Optimizing Performance [30 pts]
As before, we will be using [Kaggle](https://www.kaggle.com/t/293ccc99ce8b4b638c26fb46800a762e) with held out data to test the performance of our model.

For this homework, *you will be graded on your ability to pass perfomance benchmarks*. To receive full credit for this section of the assignment you will need to pass 3 benchmarks (baseline, easy, moderate) ***on both the public and private leaderboard***. Remember that since the private leaderboard is a part of your grade, you want to refrain from solely overfitting to the public leaderboard! Partial credit will be allotted depending on how many benchmarks are passed.


The top three leaders on the private leaderboard will recieve extra credit (if there are ties, everyone tied will receive the same number of points):
* 5 points for first place
* 3 points for second place
* 2 points for third place


The following Kaggle notes from the previous assignments still apply:
* You can use any team name (the name that will show up on the Kaggle leaderboard) as long as it is not inappropriate or offensive; however, in order to receive credit, you **must** specify your `team name` in your notebook here. If you do not, there is no way for us to assign you credit!
* Kaggle lists the close date as several days after the homework's due date. This is because Kaggle does not support late submissions. The homework and your submission on Kaggle are due by the due date listed here, but you may use late days and turn it in late (i.e., if you submit Kaggle predictions after the due date, it will automatically count towards your late days even if you have turned in your notebook already).
* This portion of the assignment **must** be completed independently. You cannot share prediction code or predictions with each other. In fact, you must put the exact code you use for your final predictions below. Violations will result in point deductions.
* Related, you cannot modify your prediction files manually. Violations will result in point deductions.
* **Unlike previous assignments**, you are free to use *any* classification models you would like for this kaggle competition!

### Some helpful tips:

* You are not required to use all of training data for this challenge: strategically applying filters and/or transformations (on observations or features, or both) can be helpful
* Remember that you are free to use *any* regression model you would like, regardless of whether we discussed it in class---the world is your oyster!
* As always, you want to check if your model is robust to different validation sets so that you are not overfitting to the public leaderboard.

**Kaggle team name:** `[fill in here]`

Read in the test dataset (`home_data_test.csv` and load associated images if you are using them).

In [ ]:
# TODO: put all code needed (including preprocessing steps) to make your
# final kaggle submission; note that this code must match the predictions
# that you provide on kaggle


You can see details about the file format for submission on kaggle (`sample_submission.csv`, essentially a 2 column file with `houseid`, the unique identifier in your test set, and `price`, your predictions). To make things easier, we provide here some sample code that you can modify to make your own submission file if your predictions were in a variable called `y_pred_kagg`. On Kaggle, the performance metric is RMSE as it was in Part 2 of this assignment.

In [ ]:
results = pd.Series(y_pred_kagg.flatten(), name="price")
results = pd.concat([df_test['houseid'], results], axis=1)
results.to_csv('my_submission.csv', index=False)

Once you output your csv file, you need to download the file from colab to your local computer (you can click the file folder icon on the left panel to see the files in your workspace) and upload that file to the Kaggle site as your submission. Note that you can submit multiple times (up to 10 times a day)!

## To Submit
Please provide a Google Colab link (enable `Viewer` permissions to `Rice University` only) by clicking the `Share` button and toggling permissions accordingly and pasting the link here (in markdown, within the parantheses): [Colab notebook](https://)

Now export the notebook as a PDF (`File > Print > Save as PDF`).

Make sure your pdf reflects your full notebook. Google Colab can sometimes export the notebook strangely (see [GitHub Issue](https://github.com/googlecolab/colabtools/issues/4357)).

If you run into substantial issues, it may be easier to convert your notebook to an html file and print directly from your browser. With your Google Drive already mounted, first you need to navigate to the folder with the file:

In [ ]:
# this file path assumes the default location colab notebooks are stored on drive
# if you use folders or put them in other directories, you will need to adjust the file path
%cd /content/drive/MyDrive/Colab\ Notebooks/
!ls

In [ ]:
# the ls command should have shown you your notebook, you can now replace the name of the
# notebook file in the below command
!jupyter nbconvert --to html netid-hw6.ipynb

# after this commmand, you should have a netid-hw6.html file in your Google Drive
# you can now download that file, open it in your browser and directly save as pdf

Make sure your pdf file is named `netid-hw6.pdf`, and upload it to the corresponding Gradescope assignment.

Also, double check that your Kaggle submission shows up on the public leaderboard.